In [1]:
import pandas as pd

In [2]:
dataset= pd.read_csv('data/data.csv')

# I. Helping Steps to get through the implementation
### - Reading the dataset
### - Visualizing Uncleaned Data from the dataset.
### - Acknowledging the Labels
### - Acknowledging the Features

### - Cleaning the data
### - Building the Pipeline to clean the data from the dataset.
### - Visualizing cleaned data from the dataset.

## Features/Labels
### - labels | 1/0
### - 5 numerical features
### - 8 categorical features

In [5]:
dataset.head(50)

,age,workSector,education,educationNum,statusMarriage,career,relationship,race,sex,gainedCapital,lostCapital,hoursPerWeek,country,income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
